In [1]:
from dotenv import load_dotenv
import importlib
import os
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage, SystemMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from typing import Annotated, TypedDict


MODEL = "mistral-nemo"
TEMPERATURE = 0
load_dotenv('../.env', override=True)

from research_agent import create_agent
agent = create_agent(
    model=MODEL,
    builtin_tools=['wikipedia', 'web_search', 'calculator', 'wolframalpha']
)

In [2]:
claim = "2 raised to the 10th power is 1024."
results = agent.invoke({'claim': claim})

In [5]:
for message in results['messages']:
    message.pretty_print()

================================ System Message ================================

You are an **Evidence Retrieval Agent**. Your role is to retrieve supporting or contradictory evidence for a given claim using the tools available to you.  

### **Rules:**  
1. **Do Not Evaluate the Claim** – Your task is to retrieve evidence, not to judge the claim's truth.  
2. **Select and Use Tools Strategically** – Based on the claim, think carefully which tools would help, and construct the correct tool calls to retrieve relevant evidence.  
3. **Return Only Tool Messages** – Your output should consist solely of tool calls with properly structured arguments. Do not generate natural language explanations.  
4. **Handle Ambiguity Thoughtfully** – If the claim is unclear, attempt a best-guess search or request clarification through a tool call if applicable.  
5. **Retrieve Evidence from Multiple Perspectives** – If conflicting evidence exists, return both supporting and contradictory sources.  

### 

The tool calls are in AI Messages. That is, an `AIMessage` will have a `.tool_calls` attribute which is a list of tool calls or `None`

In [10]:
from pprint import pprint
print(type(results['messages'][2]))
pprint(results['messages'][2].tool_calls)

<class 'langchain_core.messages.ai.AIMessage'>
[{'args': {'expression': '2 ** 10'},
  'id': '421e56a4-b5f4-4de4-8feb-0b85e76221ba',
  'name': 'calculator',
  'type': 'tool_call'}]


In [13]:
print(results['evidence'])

[{'name': 'calculator', 'args': {'expression': '2 ** 10'}, 'result': '1024'}]


In [5]:
# from langchain_core.messages import HumanMessage, SystemMessage
claim = "1/3 is bigger than 1/4."
# initial_state = {"claim": claim}
# final_state = graph.invoke(initial_state)

messages = [sys_msg]
messages = react_graph.invoke({"messages": messages, "claim": claim})

In [6]:
for m in messages['messages']:
    m.pretty_print()

================================ System Message ================================

You are an **Evidence Retrieval Agent**. Your role is to retrieve supporting or contradictory evidence for a given claim using the tools available to you.  

### **Rules:**  
1. **Do Not Evaluate the Claim** – Your task is to retrieve evidence, not to judge the claim's truth.  
2. **Select and Use Tools Strategically** – Based on the claim, think carefully which tools would help, and construct the correct tool calls to retrieve relevant evidence.  
3. **Return Only Tool Messages** – Your output should consist solely of tool calls with properly structured arguments. Do not generate natural language explanations.  
4. **Handle Ambiguity Thoughtfully** – If the claim is unclear, attempt a best-guess search or request clarification through a tool call if applicable.  
5. **Retrieve Evidence from Multiple Perspectives** – If conflicting evidence exists, return both supporting and contradictory sources.  

### 

In [7]:
from pprint import pprint
pprint(messages)

{'claim': '1/3 is bigger than 1/4.',
 'evidence': [{'args': {'query': 'Compare the sizes of fractions 1/3 and 1/4.'},
               'name': 'query',
               'result': "No Wikipedia page found for 'Compare the sizes of "
                         "fractions 1/3 and 1/4.'."}],
 'messages': [SystemMessage(content="You are an **Evidence Retrieval Agent**. Your role is to retrieve supporting or contradictory evidence for a given claim using the tools available to you.  \n\n### **Rules:**  \n1. **Do Not Evaluate the Claim** – Your task is to retrieve evidence, not to judge the claim's truth.  \n2. **Select and Use Tools Strategically** – Based on the claim, think carefully which tools would help, and construct the correct tool calls to retrieve relevant evidence.  \n3. **Return Only Tool Messages** – Your output should consist solely of tool calls with properly structured arguments. Do not generate natural language explanations.  \n4. **Handle Ambiguity Thoughtfully** – If the claim i

ToolMessage(content="No Wikipedia page found for 'Compare the sizes of fractions 1/3 and 1/4.'.", name='query', id='dd7e1db8-9f8d-412e-96d6-bf28b3aec30b', tool_call_id='e5d05507-9826-4647-9c62-0485a24e75c5')

In [14]:
response = react_graph.nodes['assistant'].invoke(
    {"messages": [HumanMessage(content="hi there")]}
)

In [15]:
print(response)

{'messages': AIMessage(content="Hello! How can I help you today? Let's have a nice conversation. If you need any calculations, I can do that too! 😊", additional_kwargs={}, response_metadata={'model': 'mistral-nemo', 'created_at': '2025-03-15T20:58:08.533286Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4515353959, 'load_duration': 617177417, 'prompt_eval_count': 263, 'prompt_eval_duration': 3137000000, 'eval_count': 32, 'eval_duration': 759000000, 'message': Message(role='assistant', content="Hello! How can I help you today? Let's have a nice conversation. If you need any calculations, I can do that too! 😊", images=None, tool_calls=None)}, id='run-5b14021e-fbe7-4626-ba5c-aee5d59ba529-0', usage_metadata={'input_tokens': 263, 'output_tokens': 32, 'total_tokens': 295})}
